# Crypto Agent with Real-time News (built using Financial Modeling Prep and LlamaIndex)
- Testing between OpenAI o3-mini and Anthropic Claude 3.5 Sonnet

## 1. Process <br>
**Crypto Analysis Agent**<br>

Overview

This notebook implements an intelligent crypto analysis system that combines reasoning or chat models, LlamaIndex for orchestration, and Financial Modeling Prep (FMP) API for real-time cryptocurrency data and news. The agent can perform sophisticated market analysis, sentiment evaluation, and provide trading insights using natural language queries.

### Tools used:

1.   OpenAI API - o3-mini. Get your OpenAI API [here](https://platform.openai.com/)
2.   Anthropic API - Claude 3.5 Sonnet. Get your Claude API [here](https://console.anthropic.com/)
3.   LlamaIndex for function calling orchestration
4.   Financial Modeling Prep API for stock information and news. Get a free API key [here](https://site.financialmodelingprep.com/)



In [1]:
!pip install llama-index-llms-anthropic -q
!pip install llama-index -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.8/222.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 19.7 MB/s eta 0:00:00


In [2]:
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool

import nest_asyncio

nest_asyncio.apply()

## 2. Instantiate Anthropic and News API Keys

**API Key Configuration**

Instantiate Baseline LLM to be used

Store your API keys in Google Colab's user data:

ANTHROPIC_API_KEY: For Claude access
<br>FINANCIAL_MODELING_PREP_API_KEY: For market data access


*   Experiment 1: Use Claude Sonnet for tool use, code generation and parsing
*   Optional: Users can swap out Claude for OpenAI models to test different models and evaluate different provider outputs / save costs



In [3]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
CLAUDE_API_KEY = userdata.get('ANTHROPIC_API_KEY')
FMP_API_KEY = userdata.get('FMP_API_KEY')

In [4]:
anthropic_llm = Anthropic(model="claude-3-5-sonnet-latest", api_key=CLAUDE_API_KEY)

In [5]:
openai_llm = OpenAI(model="o3-mini", api_key=OPENAI_API_KEY)

## 3. Define Classes for Chaining Functions

Class Structure

* CryptoMetrics: Data class for storing cryptocurrency metrics
* HistoricalPrice: Data class for historical price data
* CryptoAnalyzer: Main class handling API interactions and data processing



In [6]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union, Any
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta

from google.colab import userdata
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import FunctionCallingAgent

@dataclass
class CryptoMetrics:
    """Data class to store key cryptocurrency metrics"""
    symbol: str
    name: str
    price: float
    changes_percentage: float
    change: float
    day_low: float
    day_high: float
    year_high: float
    year_low: float
    market_cap: float
    moving_avg_50: float
    moving_avg_200: float
    volume: int
    avg_volume: int
    exchange: str
    open_price: float
    previous_close: float
    shares_outstanding: int
    timestamp: int

@dataclass
class HistoricalPrice:
    """Data class to store historical price data"""
    date: str
    open: float
    high: float
    low: float
    close: float
    adj_close: float
    volume: int
    unadjusted_volume: int
    change: float
    change_percent: float
    vwap: float
    label: str
    change_over_time: float

class CryptoAnalyzer:
    def __init__(self):
        """
        Initialize the Crypto analyzer using Google Colab userdata for API key.
        """
        self.api_key = userdata.get('FMP_API_KEY')
        if not self.api_key:
            raise ValueError("FMP API key not found in Colab userdata")

        self.base_url = "https://financialmodelingprep.com/api/v3"
        self.pro_url = "https://financialmodelingprep.com/api/v4"
        self.df = None
        self._fetch_data()

    def _fetch_data(self) -> None:
        """
        Fetch fresh cryptocurrency data from the FMP API and update the internal dataframe.
        """
        url = f"{self.base_url}/quotes/crypto?apikey={self.api_key}"
        try:
            response = requests.get(url)
            response.raise_for_status()
            self.raw_data = response.json()
            self.df = pd.DataFrame(self.raw_data)
            # Convert timestamp to datetime
            self.df['timestamp'] = pd.to_datetime(self.df['timestamp'], unit='s')
        except requests.exceptions.RequestException as e:
            raise Exception(f"Failed to fetch crypto data: {str(e)}")
        except Exception as e:
            raise Exception(f"Error processing crypto data: {str(e)}")

    def refresh_data(self) -> None:
        """
        Manually refresh the cryptocurrency data from the API.
        """
        self._fetch_data()

    def search_crypto(self, query: str) -> List[Dict]:
        """
        Search for cryptocurrencies by symbol or name.
        Returns a list of matching cryptocurrencies.
        """
        if self.df is None:
            self._fetch_data()

        # Convert query and dataframe columns to lowercase for case-insensitive search
        query = query.lower()
        matches = self.df[
            self.df['symbol'].str.lower().str.contains(query) |
            self.df['name'].str.lower().str.contains(query)
        ]

        return matches.to_dict('records')

    def get_crypto_metrics(self, identifier: str) -> Dict:
        """
        Get detailed cryptocurrency metrics by symbol or name.
        """
        try:
            if self.df is None:
                self._fetch_data()

            # Try to find by symbol first
            crypto_data = self.df[self.df['symbol'].str.lower() == identifier.lower()]

            # If not found by symbol, try to find by name
            if crypto_data.empty:
                crypto_data = self.df[self.df['name'].str.lower() == identifier.lower()]

            if crypto_data.empty:
                return {"error": f"No cryptocurrency found for: {identifier}"}

            crypto_data = crypto_data.iloc[0]

            return {
                "symbol": crypto_data['symbol'],
                "name": crypto_data['name'],
                "price": crypto_data['price'],
                "changes_percentage": crypto_data['changesPercentage'],
                "change": crypto_data['change'],
                "day_low": crypto_data['dayLow'],
                "day_high": crypto_data['dayHigh'],
                "year_high": crypto_data['yearHigh'],
                "year_low": crypto_data['yearLow'],
                "market_cap": crypto_data['marketCap'],
                "moving_avg_50": crypto_data['priceAvg50'],
                "moving_avg_200": crypto_data['priceAvg200'],
                "volume": crypto_data['volume'],
                "avg_volume": crypto_data['avgVolume'],
                "exchange": crypto_data['exchange'],
                "open_price": crypto_data['open'],
                "previous_close": crypto_data['previousClose'],
                "shares_outstanding": crypto_data['sharesOutstanding'],
                "timestamp": crypto_data['timestamp']
            }
        except Exception as e:
            return {"error": f"Failed to get crypto metrics: {str(e)}"}

    def get_historical_prices(
        self,
        symbol: str,
        limit: Optional[int] = None,
        from_date: Optional[str] = None,
        to_date: Optional[str] = None
    ) -> Dict:
        """
        Get historical price data for a specific cryptocurrency.

        Args:
            symbol: The cryptocurrency symbol (e.g., 'BTCUSD')
            limit: Number of historical data points to retrieve (optional)
            from_date: Start date in 'YYYY-MM-DD' format (optional)
            to_date: End date in 'YYYY-MM-DD' format (optional)

        Returns:
            Dictionary containing symbol and list of historical price data
        """
        try:
            # Build URL based on provided parameters
            base_url = f"{self.base_url}/historical-price-full/{symbol}?apikey={self.api_key}"

            # Add date range parameters if provided
            if from_date and to_date:
                # Validate date formats
                try:
                    datetime.strptime(from_date, '%Y-%m-%d')
                    datetime.strptime(to_date, '%Y-%m-%d')
                except ValueError:
                    return {"error": "Invalid date format. Please use YYYY-MM-DD"}

                base_url += f"&from={from_date}&to={to_date}"
            elif limit:
                base_url += f"&limit={limit}"

            response = requests.get(base_url)
            response.raise_for_status()
            data = response.json()

            if not data or 'historical' not in data:
                return {"error": f"No historical data found for symbol: {symbol}"}

            # Convert historical data to HistoricalPrice objects
            historical_prices = []
            for entry in data['historical']:
                historical_prices.append(HistoricalPrice(
                    date=entry['date'],
                    open=entry['open'],
                    high=entry['high'],
                    low=entry['low'],
                    close=entry['close'],
                    adj_close=entry['adjClose'],
                    volume=entry['volume'],
                    unadjusted_volume=entry['unadjustedVolume'],
                    change=entry['change'],
                    change_percent=entry['changePercent'],
                    vwap=entry['vwap'],
                    label=entry['label'],
                    change_over_time=entry['changeOverTime']
                ))

            return {
                "symbol": data['symbol'],
                "historical": historical_prices,
                "period": {
                    "from": from_date if from_date else historical_prices[-1].date,
                    "to": to_date if to_date else historical_prices[0].date
                }
            }

        except requests.exceptions.RequestException as e:
            return {"error": f"API request failed: {str(e)}"}
        except Exception as e:
            return {"error": f"Failed to get historical prices: {str(e)}"}

### Helper Functions for Agent Use

### Core Functionality

**Data Retrieval Functions**

1. `get_crypto_data(symbol)`: Fetches current metrics for a cryptocurrency
2. `get_historical_crypto_data(symbol, limit, from_date, to_date)`: Retrieves historical price data
3. `get_crypto_news(symbol)`: Fetches recent news and press releases (FMP Pro Subscription required)
<br>

**Analysis Capabilities**

* Real-time price and volume metrics
* Historical price analysis
* Market sentiment analysis through news
* Comparative analysis between cryptocurrencies
* Custom valuation frameworks

#### Retrieve Quote and Profile Info - get_crypto_data




In [7]:
# Helper Functions for Tool Use
def get_crypto_data(symbol: str) -> Dict:
    """
    Get detailed cryptocurrency metrics including price, volume, and market data.
    """
    try:
        analyzer = CryptoAnalyzer()
        return analyzer.get_crypto_metrics(symbol)
    except Exception as e:
        return {"error": f"Failed to get crypto metrics: {str(e)}"}

In [8]:
get_crypto_data("SOLUSD")

{'symbol': 'SOLUSD',
 'name': 'Solana USD',
 'price': 184.01,
 'changes_percentage': -2.35933,
 'change': -4.44631,
 'day_low': 181.11913,
 'day_high': 189.45515,
 'year_high': 294.33496,
 'year_low': 98.58521,
 'market_cap': 85777136219.0,
 'moving_avg_50': 212.23822,
 'moving_avg_200': 183.40755,
 'volume': 3293642752.0,
 'avg_volume': 5871799404.0,
 'exchange': 'CRYPTO',
 'open_price': 188.45631,
 'previous_close': 188.45631,
 'shares_outstanding': 466154753.650996,
 'timestamp': Timestamp('2025-02-17 14:47:16')}

#### Get Historical Data - get_historical_crypto_data

In [9]:
def get_historical_crypto_data(
    symbol: str,
    limit: int = 2,
    from_date: Optional[str] = None,
    to_date: Optional[str] = None
) -> Dict:
    """
    Get historical price data for a specific cryptocurrency.

    Args:
        symbol: The cryptocurrency symbol (e.g., 'BTCUSD')
        limit: Number of historical data points to retrieve (optional)
        from_date: Start date in 'YYYY-MM-DD' format (optional)
        to_date: End date in 'YYYY-MM-DD' format (optional)

    Example:
        # Get last 30 days of data
        btc_history = get_historical_crypto_data("BTCUSD", limit=30)

        # Get data for specific date range
        btc_history = get_historical_crypto_data(
            "BTCUSD",
            from_date="2023-08-10",
            to_date="2023-09-10"
        )
    """
    try:
        analyzer = CryptoAnalyzer()

        # If from_date is not provided, calculate it for the last year
        if from_date is None:
            today = datetime.today()
            one_year_ago = today - timedelta(days=365)
            from_date = one_year_ago.strftime("%Y-%m-%d")

        return analyzer.get_historical_prices(symbol, limit, from_date, to_date)
    except Exception as e:
        return {"error": f"Failed to get historical prices: {str(e)}"}

In [12]:
## Test Function

get_historical_crypto_data("BTCUSD", 1, "2025-01-01", "2025-02-01")

{'symbol': 'BTCUSD',
 'historical': [HistoricalPrice(date='2025-02-01', open=102414.05, high=102781.65, low=100250, close=100623.85, adj_close=100623.85, volume=27757944848, unadjusted_volume=27757944848, change=-1790.2, change_percent=-1.748, vwap=101517.3875, label='February 01, 25', change_over_time=-0.01748),
  HistoricalPrice(date='2025-01-31', open=104742.63, high=106090, low=101506, close=102411.26, adj_close=102411.26, volume=45732764360, unadjusted_volume=45732764360, change=-2331.37, change_percent=-2.22581, vwap=103687.4725, label='January 31, 25', change_over_time=-0.0222581),
  HistoricalPrice(date='2025-01-30', open=103747.25, high=106484.77, low=103289.74, close=104742.64, adj_close=104742.64, volume=44309655552, unadjusted_volume=44309655552, change=995.39, change_percent=0.95944, vwap=104566.1, label='January 30, 25', change_over_time=0.0095944),
  HistoricalPrice(date='2025-01-29', open=101290.01, high=104829.64, low=101275.6, close=103747.25, adj_close=103747.25, vol

#### Get Latest Crypto News - get_crypto_news (FMP Pro Version required)

In [13]:
def get_crypto_news(symbol: str) -> Dict:
    """
    Get recent company news and press releases.
    """
    try:
        analyzer = CryptoAnalyzer()
        url = f"{analyzer.pro_url}/crypto_news?symbol={symbol}&limit=5&apikey={analyzer.api_key}"

        response = requests.get(url)
        news = response.json()

        if not news:
            return {"error": "No news found"}

        formatted_news = [{
            "title": item.get('title'),
            "date": item.get('publishedDate'),
            "source": item.get('site'),
            "url": item.get('url'),
            "summary": item.get('text'),
        } for item in news]

        return {"news": formatted_news}
    except Exception as e:
        return {"error": f"Failed to get company news: {str(e)}"}

In [14]:
## Test function

get_crypto_news("ETHUSD")

{'news': [{'title': "Burn address receives 500 ETH and cryptic ‘brain-computer weapon' message",
   'date': '2025-02-17T14:18:53.000Z',
   'source': 'crypto.news',
   'url': 'https://crypto.news/burn-address-receives-500-eth-and-cryptic-brain-computer-weapon-message/',
   'summary': "An unknown sender transferred 500 ETH to a burn address along with a cryptic message that accuses ‘Kuande Investment' CEOs of brain-washing employees."},
  {'title': 'Crypto Price Analysis 2-17: BITCOIN: BTC, ETHEREUM: ETH, SOLANA: SOL, DOGWIFHAT: WIF, CELESTIA: TIA, ARBITRUM: ARB, OPTIMISM: OP',
   'date': '2025-02-17T14:14:53.000Z',
   'source': 'cryptodaily.co.uk',
   'url': 'https://cryptodaily.co.uk/2025/02/crypto-price-analysis-2-17-bitcoin-btc-ethereum-eth-solana-sol-dogwifhat-wif-celestia-tia-arbitrum-arb-optimism-op',
   'summary': 'Bitcoin (BTC) has started the week in the red, with the price down over 1% and trading considerably below the $97,000 level at $96,400. The flagship currency dipped to

### Instantiate Helper Functions

---



In [15]:
# Create Function Tools
## Comment out tools that are not relevant, chain different tools as needed

tools = {
    "crypto_metrics": FunctionTool.from_defaults(fn=get_crypto_data),
    # "historical_crypto_data": FunctionTool.from_defaults(fn=get_historical_crypto_data),
    "crypto_news": FunctionTool.from_defaults(fn=get_crypto_news)
}

## 4. Define Agent Flow

### Create an Agent

> Add blockquote



In [16]:
# Create the Agent
def create_crypto_agent(str):
    """
    Create a comprehensive crypto analysis agent with all available tools.
    """
    agent = FunctionCallingAgent.from_tools(
        list(tools.values()),
        llm=anthropic_llm,
        verbose=True,
        allow_parallel_tool_calls=False,
    )
    return agent

In [17]:
# Initialize the agent
claude_crypto_agent = create_crypto_agent(anthropic_llm)

In [18]:
# Initialize the agent
oai_crypto_agent = create_crypto_agent(openai_llm)

# Chat with an Agent

### Evaluate Different LLMs

#### OpenAI's o3-mini Test

- Cost effective reasoning model (Chain of thought)
- Comes with tool use and function calling support

Read more here: https://openai.com/index/openai-o3-mini/

In [19]:
# Make queries
response = oai_crypto_agent.chat("Analyze the key metrics of FETUSD")
print(str(response))

> Running step 3bd8985d-42c3-4217-9f0e-5e6d43c39715. Step input: Analyze the key metrics of FETUSD
Added user message to memory: Analyze the key metrics of FETUSD
=== LLM Response ===
I'll help you analyze the key metrics for FETUSD using the get_crypto_data function.
=== Calling Function ===
Calling function: get_crypto_data with args: {"symbol": "FETUSD"}
=== Function Output ===
{'symbol': 'FETUSD', 'name': 'Fetch.ai USD', 'price': 0.7989, 'changes_percentage': 2.52473, 'change': 0.01967334, 'day_low': 0.77074, 'day_high': 0.8117, 'year_high': 3.47427, 'year_low': 0.62082, 'market_cap': 653969927.0, 'moving_avg_50': 1.12481, 'moving_avg_200': 1.29666, 'volume': 108838368.0, 'avg_volume': 354972734.0, 'exchange': 'CRYPTO', 'open_price': 0.77923, 'previous_close': 0.77923, 'shares_outstanding': 818587968.0, 'timestamp': Timestamp('2025-02-17 14:51:29')}
> Running step 29a5b15c-1896-4c28-b3d2-a248fd782b27. Step input: None
=== LLM Response ===
Let me break down the key metrics for Fetch

In [20]:
# Make queries
response = oai_crypto_agent.chat("Analyze the price trends for ETHUSD and determine what's driving the price", chat_history=[])  # Reset chat history
print(str(response))

> Running step d65e7e53-de4d-4df4-bb2e-36ca84790182. Step input: Analyze the price trends for ETHUSD and determine what's driving the price
Added user message to memory: Analyze the price trends for ETHUSD and determine what's driving the price
=== LLM Response ===
I'll help you analyze ETH/USD price trends and their drivers. Let me gather both the price data and recent news that might be affecting the price.
=== Calling Function ===
Calling function: get_crypto_data with args: {"symbol": "ETHUSD"}
=== Function Output ===
{'symbol': 'ETHUSD', 'name': 'Ethereum USD', 'price': 2819.78, 'changes_percentage': 5.91808, 'change': 157.5527, 'day_low': 2650.8394, 'day_high': 2825.4, 'year_high': 4106.9556, 'year_low': 2122.5461, 'market_cap': 339215903806.0, 'moving_avg_50': 3139.2578, 'moving_avg_200': 2936.5308, 'volume': 17918881792.0, 'avg_volume': 31486209221.0, 'exchange': 'CRYPTO', 'open_price': 2662.2273, 'previous_close': 2662.2273, 'shares_outstanding': 120298712.596962, 'timestamp':

#### Anthropic Claude Sonnet 3.5 Test

- Versatile and powerful Claude model
- Observed better performance when working with structured data sets
- Intensive token usage and cost per million token

In [21]:
def create_crypto_agent(anthropic_llm):
    """
    Create a comprehensive crypto analysis agent with all available tools.
    """
    agent = FunctionCallingAgent.from_tools(
        list(tools.values()),
        llm=anthropic_llm,
        verbose=True,
        allow_parallel_tool_calls=False,
    )
    return agent

# Initialize the agent
crypto_agent = create_crypto_agent(anthropic_llm)

In [22]:
response = crypto_agent.chat("Analyze Polygon's fundamentals and provide sentiment analysis around recent news")
print(str(response))

> Running step 847a373b-c7c3-48fa-8da9-a6a7942e50b9. Step input: Analyze Polygon's fundamentals and provide sentiment analysis around recent news
Added user message to memory: Analyze Polygon's fundamentals and provide sentiment analysis around recent news
=== LLM Response ===
I'll help you analyze Polygon (MATIC) by getting both its fundamental data and recent news. I'll use both available tools to provide a comprehensive analysis.
=== Calling Function ===
Calling function: get_crypto_data with args: {"symbol": "MATIC"}
=== Function Output ===
{'error': 'No cryptocurrency found for: MATIC'}
> Running step 015327cc-ebf8-44ac-ad3b-01fb0aae0011. Step input: None
=== Calling Function ===
Calling function: get_crypto_news with args: {"symbol": "MATIC"}
=== Function Output ===
{'news': [{'title': 'MATIC Set For Uptick: Analyst Eyes $1.30 In the Coming Weeks', 'date': '2024-05-20T15:39:37.000Z', 'source': 'https://tronweekly.com', 'url': 'https://www.tronweekly.com/matic-set-for-uptick-analy

In [ ]:
query= "Act like a value investor (such as Warren Buffett or Charlie Munger) and assess the financial health of Shibu Ina Coin"
response = crypto_agent.chat(query)
print(str(response))

> Running step 876da26b-7f2a-4cce-bbca-9bd1d5f8c4aa. Step input: Act like a value investor (such as Warren Buffett or Charlie Munger) and assess the financial health of Shibu Ina Coin
Added user message to memory: Act like a value investor (such as Warren Buffett or Charlie Munger) and assess the financial health of Shibu Ina Coin
=== LLM Response ===
I'll analyze Shiba Inu (SHIB) from a value investing perspective, focusing on fundamentals and intrinsic value as Warren Buffett and Charlie Munger would.
=== Calling Function ===
Calling function: get_crypto_data with args: {"symbol": "SHIB"}
=== Function Output ===
{'error': 'No cryptocurrency found for: SHIB'}
> Running step 80cbf0c0-da25-4297-8267-975c337b2c20. Step input: None
=== Calling Function ===
Calling function: get_crypto_news with args: {"symbol": "SHIB"}
=== Function Output ===
{'news': [{'title': 'Shiba Inu’s Insane $0.001 Price Prediction: Will It Create a New Wave of Millionaires?', 'date': '2024-05-20T17:47:22.000Z', 's

## Evaluate LLM Outputs (o3-mini vs. Claude Sonnet 3.5)

In [ ]:
query= "Create a valuation framework for SOLUSD (use a step-by-step logic to deduce a consistent method of valuing crypto) and present your results in a table format"
response = claude_crypto_agent.chat(query)
print(str(response))

> Running step 0eb6f73e-04f3-4518-a023-87595fde373e. Step input: Create a valuation framework for SOLUSD (use a step-by-step logic to deduce a consistent method of valuing crypto) and present your results in a table format
Added user message to memory: Create a valuation framework for SOLUSD (use a step-by-step logic to deduce a consistent method of valuing crypto) and present your results in a table format
=== LLM Response ===
I'll help create a valuation framework for SOL/USD using available data. Let me gather the current metrics first.
=== Calling Function ===
Calling function: get_crypto_data with args: {"symbol": "SOLUSD"}
=== Function Output ===
{'symbol': 'SOLUSD', 'name': 'Solana USD', 'price': 195.95, 'changes_percentage': 0.80974, 'change': 1.57395, 'day_low': 194.37605, 'day_high': 196.11, 'year_high': 294.33496, 'year_low': 98.58521, 'market_cap': 91343023977.0, 'moving_avg_50': 211.9817, 'moving_avg_200': 183.11371, 'volume': 2705406208.0, 'avg_volume': 6073631101.0, 'exc

In [ ]:
query= "Create a valuation framework for SOLUSD (use a step-by-step logic to deduce a consistent method of valuing crypto) and present your results in a table format"
response = oai_crypto_agent.chat(query)
print(str(response))

> Running step e174203e-6c34-40f3-a8b6-4d9265b79412. Step input: Create a valuation framework for SOLUSD (use a step-by-step logic to deduce a consistent method of valuing crypto) and present your results in a table format
Added user message to memory: Create a valuation framework for SOLUSD (use a step-by-step logic to deduce a consistent method of valuing crypto) and present your results in a table format
=== LLM Response ===
I'll help create a valuation framework for Solana (SOL) by first gathering the current data and then developing a structured approach.
=== Calling Function ===
Calling function: get_crypto_data with args: {"symbol": "SOLUSD"}
=== Function Output ===
{'symbol': 'SOLUSD', 'name': 'Solana USD', 'price': 195.7467, 'changes_percentage': 0.70515, 'change': 1.37065, 'day_low': 194.37605, 'day_high': 196.11, 'year_high': 294.33496, 'year_low': 98.58521, 'market_cap': 91248254716.0, 'moving_avg_50': 211.9817, 'moving_avg_200': 183.11371, 'volume': 2695214080.0, 'avg_volu

## Simple Eval Framework

In [23]:
import time
from difflib import SequenceMatcher
import pandas as pd

# Example list of test queries.
# For some queries, an "expected" answer is provided (if available) for accuracy evaluation.
test_queries = [
    {
        "query": "Provide a concise financial risk analysis for Solana (SOLUSD) based on market metrics.",
        "expected": "Expected risk analysis details..."
    }
]

# Simulated agents – in your code these would be your actual LLM agent interfaces.
# For demonstration, we define stubs that return a dummy response.
def create_crypto_agent(llm):
    """
    Create a comprehensive crypto analysis agent with all available tools.
    """
    agent = FunctionCallingAgent.from_tools(
        list(tools.values()),
        llm=llm,
        verbose=True,
        allow_parallel_tool_calls=False,
    )
    return agent

# Assume anthropic_llm and openai_llm, as well as 'tools' and FunctionCallingAgent are defined.
claude_agent = create_crypto_agent(anthropic_llm)
openai_agent = create_crypto_agent(openai_llm)

def send_query(agent, query):
    """Send the query to the agent and measure latency."""
    start_time = time.time()
    response = agent.chat(query)
    latency = time.time() - start_time
    return response, latency

def evaluate_accuracy(response, expected):
    """
    Compute a similarity score between the LLM response and the expected output.
    Returns a ratio between 0 (no match) and 1 (perfect match).
    """
    return SequenceMatcher(None, response, expected).ratio()

def evaluate_response_quality(response):
    """
    Evaluate quality based on simple heuristics.
    For example, check if the response is structured (contains a table-like format)
    and includes numerical data.
    Returns a quality score between 0 and 1.
    """
    response_str = str(response)  # ensure we have a string
    score = 0.0
    # Check for table formatting (e.g. pipe '|' characters or "table" keyword).
    if "|" in response_str or "table" in response_str.lower():
        score += 0.5
    # Check for presence of numeric values (financial figures).
    if any(char.isdigit() for char in response_str):
        score += 0.5
    return score

def evaluate_data_handling(response):
    """
    Evaluate data handling by checking for error indicators or format issues.
    Returns 1 if the response appears to properly handle data, 0 otherwise.
    """
    response_str = str(response)
    return 0 if "error" in response_str.lower() or "exception" in response_str.lower() else 1

# Container to store results for each query and each agent.
evaluation_results = {}

for test in test_queries:
    query_text = test["query"]
    expected = test.get("expected")  # may be None if no expected answer is provided
    evaluation_results[query_text] = {}

    for agent_name, agent in [("Claude", claude_agent), ("OpenAI", openai_agent)]:
        response, latency = send_query(agent, query_text)
        # Convert response to string once for all evaluations.
        response_str = str(response)
        accuracy = evaluate_accuracy(response_str, expected) if expected else None
        quality = evaluate_response_quality(response_str)
        data_handling = evaluate_data_handling(response_str)

        evaluation_results[query_text][agent_name] = {
            "response": response_str,
            "latency_sec": latency,
            "accuracy": accuracy,
            "quality": quality,
            "data_handling": data_handling
        }

# Package the evaluation results into a variable for later use.
evaluation_report = evaluation_results

> Running step 7d70f642-73dc-4908-b845-e1b81e30c3cd. Step input: Provide a concise financial risk analysis for Solana (SOLUSD) based on market metrics.
Added user message to memory: Provide a concise financial risk analysis for Solana (SOLUSD) based on market metrics.
=== LLM Response ===
I'll help you analyze Solana's financial risk profile using market data. Let me fetch the current metrics.
=== Calling Function ===
Calling function: get_crypto_data with args: {"symbol": "SOLUSD"}
=== Function Output ===
{'symbol': 'SOLUSD', 'name': 'Solana USD', 'price': 183.7, 'changes_percentage': -2.52383, 'change': -4.75631, 'day_low': 181.11913, 'day_high': 189.45515, 'year_high': 294.33496, 'year_low': 98.58521, 'market_cap': 85632628245.0, 'moving_avg_50': 212.23822, 'moving_avg_200': 183.40755, 'volume': 3303880448.0, 'avg_volume': 5871799404.0, 'exchange': 'CRYPTO', 'open_price': 188.45631, 'previous_close': 188.45631, 'shares_outstanding': 466154753.650996, 'timestamp': Timestamp('2025-02-

In [24]:
# (Optional) Print the raw report to the console.
for query, agents in evaluation_report.items():
    print(f"\nQuery: {query}")
    for agent, metrics in agents.items():
        print(f"--- {agent} ---")
        print(f"Response: {metrics['response']}")
        print(f"Latency: {metrics['latency_sec']*1000:.2f} ms")
        if metrics['accuracy'] is not None:
            print(f"Accuracy: {metrics['accuracy']*100:.1f}%")
        print(f"Response Quality Score: {metrics['quality']:.2f}")
        print(f"Data Handling Score: {metrics['data_handling']}")



Query: Provide a concise financial risk analysis for Solana (SOLUSD) based on market metrics.
--- Claude ---
Response: Based on the market data, here's a concise risk analysis for Solana (SOL):

Volatility Risk:
- Current price ($183.70) is showing significant daily volatility with a -2.52% change
- Wide daily trading range: $181.12 (low) to $189.46 (high)
- Trading well below its 50-day moving average ($212.24), indicating recent bearish pressure

Market Position:
- Strong market cap of $85.63B suggests good liquidity and stability
- Current volume ($3.30B) is below average volume ($5.87B), indicating potentially reduced liquidity
- Trading at 62.4% of its yearly high ($294.33), with support at yearly low of $98.59

Key Risk Indicators:
1. Technical Position: Price is near the 200-day moving average ($183.41), suggesting a critical support/resistance level
2. Liquidity Risk: Below-average volume could lead to increased price volatility
3. Market Sentiment: Currently in a short-term d